In [3]:
#Import required packages
import numpy as np, xarray as xr, matplotlib.pyplot as plt, xradar as xd, pyart, pandas as pd, datetime


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



Get Severe Events

In [5]:
county_based_warnings = []

#Iterate through CSV files containing National Weather Service warning times for each county, concatenate into DataFrame
for filename in [r'C:\Users\benja\Downloads\vtec_ALC' + str(num).zfill(3) + '_20150101_20250505.csv' for num in [7, 121, 117, 115, 37, 21, 73]]:
    county_based_warnings.append(pd.read_csv(filename))
all_warnings = pd.concat(county_based_warnings)

#Select severe thunderstorm warnings, drop warnings with duplicate issuance times
all_stws = all_warnings[all_warnings['name'] == 'Severe Thunderstorm Warning'].reset_index(drop = True)
all_swts_noduplicates = all_stws.drop_duplicates(subset=['iso_issued'])


In [ ]:
#Import package to get radar scans from AWS (from https://nexradaws.readthedocs.io/en/latest/Tutorial.html#Get-available-radars-in-a-given-year,-month,-and-day)
import nexradaws
conn = nexradaws.NexradAwsInterface()

all_grid_xarrays = []
radar_objects = []
#Iterate through issuance times of severe thunderstorm warnings
for idx, timeval in enumerate(pd.to_datetime(all_swts_noduplicates['iso_issued'][0:])):
    print(idx + 0)
    try:
        #Get radar scan times within a 14 minute range of warning issuance time
        scans = conn.get_avail_scans_in_range(timeval - datetime.timedelta(minutes = 7), timeval + datetime.timedelta(minutes = 7), 'KBMX')
        filename = "s3://noaa-nexrad-level2/" + scans[0].awspath + "/" + scans[0].filename
        
        #Read first radar scan file in 14-minute range (https://arm-doe.github.io/pyart/examples/io/plot_nexrad_data_aws.html)
        radar = pyart.io.read_nexrad_archive(filename)
        
        #Grid radar scan file to 150kmx150kmx10km grid, at 400x400x11 resolution (https://arm-doe.github.io/pyart/examples/xradar/plot_grid_xradar.html#sphx-glr-examples-xradar-plot-grid-xradar-py)
        grid = pyart.map.grid_from_radars(
            (radar,),
            grid_shape=(11, 1000, 1000),
            grid_limits=(
                (0.0, 10_000),
                (-150_000.0, 150_000.0),
                (-150_000, 150_000.0),
            ), min_radius = 1000
        )

        #Convert gridded radar file into xarray, save file
        grid_xr = grid.to_xarray()
        grid_xr.to_netcdf('radar_file_' + str(idx + 0) + '.nc')
    except:
        print('Error retrieving scans for index ' + str(idx))

0
1
Error retrieving scans for index 1
2


Get Random Non-Events

In [6]:
all_swts_noduplicates.reset_index(drop=True, inplace=True)

#Get list of dates with severe thunderstorm warnings
dates_w_severe = [pd.to_datetime(all_swts_noduplicates['iso_issued'][i]).date() for i in range(len(all_swts_noduplicates))]

#Get range of hours from the first to last severe thunderstorm warning time in the dataset
dr = pd.date_range(pd.to_datetime(all_swts_noduplicates['iso_issued'][0]), 
              pd.to_datetime(all_swts_noduplicates['iso_issued'][len(all_swts_noduplicates) - 1]), 
              freq=datetime.timedelta(hours=1))

In [ ]:
idx = 0
#Iterate for as many iterations as the number of severe thunderstorm warning cases
while idx < len(all_swts_noduplicates):

    #Get random indices of dates in daterange, get associated date
    randidx = np.random.randint(len(dr))
    dateval = dr[randidx]

    #Get date from 1 hour before and 1 hour after dateval
    p1hr_date = (dateval + datetime.timedelta(hours = 1)).date()
    m1hr_date = (dateval - datetime.timedelta(hours = 1)).date()

    #If no severe thunderstorm warnings occur on days within 1 hour of selected hour..
    if (dateval.date() not in dates_w_severe) and (p1hr_date not in dates_w_severe) and (m1hr_date not in dates_w_severe): 
        try:
            #Get radar scan times within a 14 minute range of selected time
            scans = conn.get_avail_scans_in_range(dateval - datetime.timedelta(minutes = 7), dateval + datetime.timedelta(minutes = 7), 'KBMX')
            filename = "s3://noaa-nexrad-level2/" + scans[0].awspath + "/" + scans[0].filename

            #Read first radar scan file in 14-minute range
            radar = pyart.io.read_nexrad_archive(filename)

            #Grid radar scan file to 150kmx150kmx10km grid, at 400x400x11 resolution
            grid = pyart.map.grid_from_radars(
                (radar,),
                grid_shape=(11, 400, 400),
                grid_limits=(
                    (0.0, 10_000),
                    (-150_000.0, 150_000.0),
                    (-150_000, 150_000.0),
                ), min_radius = 1000
            )

            #Convert gridded radar file into xarray, save file
            grid_xr = grid.to_xarray()
            grid_xr.to_netcdf('null_radar_file_' + str(idx + 0) + '.nc')
            print('Successfully retrieved null scan with index ' + str(idx) + ' for ' + str(dateval))
            idx += 1
        except:
            print('Error retrieving scans')
    else:
        print('Selected date ' + str(dateval) + ' in severe storm date range, skipping')


Get Thunderstorm Non-Events

In [ ]:
#Read in spreadsheet containing times when thunderstorms were reported at ASOS stations within region of interest
tstm_times = pd.read_csv(r"C:\Users\benja\Downloads\tstm_times.csv", header=None)
tstm_times.columns = ['DateTime']
tstm_times = pd.to_datetime(tstm_times['DateTime'])

#Get dates on which thunderstorms occurred
tstm_dates = list(np.unique([tstm_times[i].date() for i in range(len(tstm_times))]))


In [ ]:
previous_dates = []
idx = 0
time_idx = 0
#Iterate for as many iterations as the number of severe thunderstorm warning cases
while idx < len(all_swts_noduplicates):
    #Get time of thunderstorm report, times an hour before and an hour after
    dateval = tstm_times[time_idx]
    p1hr_date = (dateval + datetime.timedelta(hours = 1)).date()
    m1hr_date = (dateval - datetime.timedelta(hours = 1)).date()
    
    #If no severe storms on date (or +- 1 hour date) and date has not already been selected, get data for selected time
    if (dateval.date() not in dates_w_severe) and (p1hr_date not in dates_w_severe) and (m1hr_date not in dates_w_severe): 
        if (dateval.date() not in previous_dates) and (p1hr_date not in previous_dates) and (m1hr_date not in previous_dates):
            try:
                #Get radar scan times within a 14 minute range of selected time
                scans = conn.get_avail_scans_in_range(dateval - datetime.timedelta(minutes = 7), dateval + datetime.timedelta(minutes = 7), 'KBMX')
                filename = "s3://noaa-nexrad-level2/" + scans[0].awspath + "/" + scans[0].filename
                
                #Read first radar scan file in 14-minute range
                radar = pyart.io.read_nexrad_archive(filename)
                
                #Grid radar scan file to 150kmx150kmx10km grid, at 400x400x11 resolution
                grid = pyart.map.grid_from_radars(
                    (radar,),
                    grid_shape=(11, 400, 400),
                    grid_limits=(
                        (0.0, 10_000),
                        (-150_000.0, 150_000.0),
                        (-150_000, 150_000.0),
                    ), min_radius = 1000
                )
    
                #Convert gridded radar file into xarray, save file
                grid_xr = grid.to_xarray()
                grid_xr.to_netcdf('tstm_nonevent_radar_file_' + str(idx + 0) + '.nc')
                print('Successfully retrieved tstm nonevent scan with index ' + str(idx) + ' for ' + str(dateval))
                previous_dates.append(dateval.date())
                idx += 1
            except:
                print('Error retrieving scans')
        else:
            print('Selected date ' + str(dateval) + ' already sampled, skipping')
    else:
        print('Selected date ' + str(dateval) + ' in severe storm date range, skipping')
    time_idx += 1

Get Rain Non-Events

In [ ]:
#Get times with rain reports at at least one ASOS station in region of interest
strat_times = pd.read_csv(r"C:\Users\benja\Downloads\stratiform_times.csv", header=None)
strat_times.columns = ['DateTime']
strat_times = pd.to_datetime(strat_times['DateTime'])


In [ ]:
previous_dates = []
idx = 0
time_idx = 0
#Iterate for as many iterations as the number of severe thunderstorm warning cases
while idx < len(all_swts_noduplicates):
    #Get time of thunderstorm report, times an hour before and an hour after
    dateval = strat_times[time_idx]
    p1hr_date = (dateval + datetime.timedelta(hours = 1)).date()
    m1hr_date = (dateval - datetime.timedelta(hours = 1)).date()
    
    #If no severe storms or thunderstorms on date (or +- 1 hour date) and date has not already been selected, get data for selected time
    if (dateval.date() not in dates_w_severe) and (p1hr_date not in dates_w_severe) and (m1hr_date not in dates_w_severe): 
        if (dateval.date() not in tstm_dates) and (p1hr_date not in tstm_dates) and (m1hr_date not in tstm_dates):
            if (dateval.date() not in previous_dates) and (p1hr_date not in previous_dates) and (m1hr_date not in previous_dates):
                try:
                    #Get radar scan times within a 14 minute range of selected time
                    scans = conn.get_avail_scans_in_range(dateval - datetime.timedelta(minutes = 7), dateval + datetime.timedelta(minutes = 7), 'KBMX')
                    filename = "s3://noaa-nexrad-level2/" + scans[0].awspath + "/" + scans[0].filename
                    
                    #Read first radar scan file in 14-minute range
                    radar = pyart.io.read_nexrad_archive(filename)
                    
                    #Grid radar scan file to 150kmx150kmx10km grid, at 400x400x11 resolution
                    grid = pyart.map.grid_from_radars(
                        (radar,),
                        grid_shape=(11, 400, 400),
                        grid_limits=(
                            (0.0, 10_000),
                            (-150_000.0, 150_000.0),
                            (-150_000, 150_000.0),
                        ), min_radius = 1000
                    )
                    
                    #Convert gridded radar file into xarray, save file
                    grid_xr = grid.to_xarray()
                    grid_xr.to_netcdf('stratiform_nonevent_radar_file_' + str(idx + 0) + '.nc')
                    print('Successfully retrieved tstm nonevent scan with index ' + str(idx) + ' for ' + str(dateval))
                    previous_dates.append(dateval.date())
                    idx += 1
                except:
                    print('Error retrieving scans')
            else:
                print('Selected date ' + str(dateval) + ' already sampled, skipping')
        else:
            print('Selected date ' + str(dateval) + ' in nonsevere thunderstorm date range, skipping')
    else:
        print('Selected date ' + str(dateval) + ' in severe storm date range, skipping')
    time_idx += 1